In [ ]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
import re

In [ ]:
url='https://www.marmiton.org/recettes/recette_tacos-mexicains_34389.aspx'

In [ ]:
r.get(url)

In [ ]:
soup=BeautifulSoup(r.get(url).content)

In [ ]:
list_of selectors=['h1','div.recipe-author__author>span','div.recipe-infos-users__rate__stars>span','ul.mrtn-tags-list>li','span.recipe-infos-users__value','div.recipe-infos,div.recipe-infos__timmings__detail','div.recipe-infos__timmings__cooking,div.recipe-infos__timmings__preparation']

In [ ]:
#Recipe Name
soup.select('h1')[0].text

In [ ]:
#Author name
soup.select('div.recipe-author__author>span')[0].text.strip('\n ')

In [ ]:
#recipe note
soup.select('div.recipe-infos-users__rate__stars>span')[0].text

In [ ]:
#tags ('entrée, 'plat unique','plats mexicains)
[i.text for i in soup.select('ul.mrtn-tags-list>li')]

In [ ]:
#Nb likes
soup.select('span.recipe-infos-users__value')[1].text

In [ ]:
#Nb of portions
[i.replace('\n',' ') for i in soup.select('div.recipe-infos,div.recipe-infos__timmings__detail')[0].text.split('\n\n') if len(i)>1]

In [ ]:
#cooking time  preparation,cuisonn
[re.sub('\t|\n|\xa0','',i.text) for i in soup.select('div.recipe-infos__timmings__cooking,div.recipe-infos__timmings__preparation')]


In [ ]:
#Ingredients
[i.text.strip().replace('\n','') for i in soup.select('li.recipe-ingredients__list__item')]

In [ ]:
#Ustensiles
[re.sub('\n\n|\t*',' ',i.text) for i in soup.select('ul.recipe-utensils__list')]

In [ ]:
[i.text for i in soup.select('ul.recipe-utensils__list')]

In [ ]:
# recipe
re.sub('\t|\n','',soup.select('ol.recipe-preparation__list')[0].text)

In [ ]:
url1='https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=50&page=4'

In [ ]:
soup1=BeautifulSoup(r.get(url1).content)

In [ ]:
[i['href'] for i in soup1.select('a.recipe-card-link')]


# Code aggregation

In [ ]:
# 14 recipes per page with page from no figure then 2 to 833
#https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=10004&page=833

In [12]:
#liste of links for the pages - start with 5pages to test before launching 833 pages...
# pb le site change de pattern de pagination 
def List_urls(NbPages):
    urls=[]
    for i in range(0,NbPages):
        url=f'https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start={4+14*i}&page={i+1}'
        urls.append(url)
        i+=1
    return urls

page_urls=List_urls(1)    
print(page_urls)

['https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=4&page=1']


In [7]:
#For one given page look for the list of recipes (links)
def find_recipe_card_link(page_urls):
    recipes_of_all_pages=[]
    for url in page_urls:
        soup=BeautifulSoup(r.get(url).content)
        # list of recipes url which are in the page
        recipes_of_all_pages.extend([f'https://www.marmiton.org'+i['href'] for i in soup.select('a.recipe-card-link')])
    return recipes_of_all_pages
        
recipes_of_all_pages=find_recipe_card_link(page_urls) 
recipes_of_all_pages

['https://www.marmiton.org/recettes/recette_original-american-cookies-de-mike_39907.aspx',
 'https://www.marmiton.org/recettes/recette_blanquette-de-veau-facile_19219.aspx',
 'https://www.marmiton.org/recettes/recette_le-crumble-aux-pommes-du-chat-qui-tousse_10534.aspx',
 'https://www.marmiton.org/recettes/recette_gateau-au-yaourt_12719.aspx',
 'https://www.marmiton.org/recettes/recette_veritable-moelleux-au-chocolat_12825.aspx',
 'https://www.marmiton.org/recettes/recette_les-timbales-de-jeanne-saumon-a-la-mousse-de-courgettes-au-micro-ondes_21864.aspx',
 'https://www.marmiton.org/recettes/recette_gateau-aux-pommes-facile_13493.aspx',
 'https://www.marmiton.org/recettes/recette_meringue-pour-les-nuls_36610.aspx',
 'https://www.marmiton.org/recettes/recette_pancakes_15299.aspx',
 'https://www.marmiton.org/recettes/recette_cookie-cup-glace-vanille-et-pepites-de-chocolat_372241.aspx',
 'https://www.marmiton.org/recettes/recette_filet-mignon-en-croute_14407.aspx',
 'https://www.marmiton.o

In [9]:
def Collect_data(recipes_of_page):
    recipe=dict() #dictionnary , 1 row of the final dataframe (Recipe name: [list of info])  recipe={'':[]}
    for url in recipes_of_page:
        list_of_infos=[]
        soup=BeautifulSoup(r.get(url).content) 
        #Recipe Name
        name=soup.select('h1')[0].text
        print(name)
        list_of_infos.append(soup.select('div.recipe-author__author>span')[0].text.strip('\n ')) # add author name
        list_of_infos.append(soup.select('div.recipe-infos-users__rate__stars>span')[0].text) # add note
        list_of_infos.append([i.text for i in soup.select('ul.mrtn-tags-list>li')])# add tags
        #add total cooking time, nb of people, Easy or not, cheap or not
        list_of_infos.extend([i.replace('\n',' ') for i in soup.select('div.recipe-infos,div.recipe-infos__timmings__detail')[0].text.split('\n\n') if len(i)>1])
        list_of_infos.append(soup.select('span.recipe-infos-users__value')[1].text) #nb likes
        #cookig and preparation times
        list_of_infos.extend([re.sub('\t|\n|\xa0','',i.text) for i in soup.select('div.recipe-infos__timmings__cooking,div.recipe-infos__timmings__preparation')])
        #ingredients
        list_of_infos.extend([i.text.strip().replace('\n','') for i in soup.select('li.recipe-ingredients__list__item')])
        #recipe (text)
        list_of_infos.extend(re.sub('\t|\n','',soup.select('ol.recipe-preparation__list')[0].text))
        recipe[name]=list_of_infos # add a row to the dictionnary . 1 row = Recipe name: [list of info]
    return 

Collect_data(recipes_of_all_pages)

Original American Cookies de Mike
Blanquette de veau facile
Le crumble aux pommes du Chat qui Tousse
Gâteau au yaourt
Véritable moelleux au chocolat
Les Timbales de Jeanne (saumon à la mousse de courgettes au micro-ondes)
Gâteau aux pommes facile
Meringue pour les nuls
Pancakes
Cookie cup glace vanille et pépites de chocolat


IndexError: list index out of range

In [ ]:
def create_dataFrame(recipe):
  pd.DataFrame(recipe)

In [ ]:
#'d4 = {}' 'for d in (d1, d2, d3): d4.update(d)'

In [ ]:
# Error Handling function

In [ ]:
# Launch function terminal

In [ ]:
url_page='https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=18&page=2'
soup=BeautifulSoup(r.get(url_page).content)
[f'https://www.marmiton.org'+i['href'] for i in soup.select('a.recipe-card-link')]

In [ ]:
https://www.marmiton.org/recettes/recette_chouquettes_30622.aspx